In [ ]:
import copy
import math
import random

import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import (
    mean_absolute_error,
    mean_absolute_percentage_error,
    mean_squared_error,
    root_mean_squared_error,
)

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader

# local scripts
from utils import (
    add_noise,
    get_train_test_splits,
    load_pems,
    split_time_series,
    TSTrainDataset,
)
from encoder.tcn_encoder import CausalCNNEncoder
from encoder.losses.info_nce import InfoNCE

### Learner

In [ ]:
class TSEncoder(nn.Module):
    def __init__(
        self,
        encoder,
        loss_fn,
        optimizer,
        train_dataset,
        lr=0.001,
        batch_size=8,
        device='cpu',
    ):
        super().__init__()
        self.device = device
        self.batch_size = batch_size

        self.net = copy.deepcopy(encoder).to(device)
        self.loss_fn = loss_fn
        self.optimizer = optimizer(self.net.parameters(), lr=lr)

        self.train_dataset = train_dataset
        self.train_dataloader = DataLoader(
            train_dataset,
            batch_size=batch_size,
            shuffle=True,
            drop_last=True,
        )

        self.n_epochs = 0
        self.n_iters = 0

    def fit(
        self,
        n_epochs=None,
        n_iters=None,
        mask_prob=0.5,
        verbose=False,
    ):
        ''' `train_ds` shape: [B, C, T]
        '''
        ### Set the default number of training iterations to 600
        if n_iters is None and n_epochs is None: n_iters = 600

        ### Training loop
        loss_log = []
        while True:
            if n_epochs is not None and self.n_epochs >= n_epochs: break

            total_loss = 0
            n_epoch_iters = 0

            interrupted = False
            ### Iterate through each mini-batch
            for x in self.train_dataloader:
                if n_iters is not None and self.n_iters >= n_iters:
                    interrupted = True
                    break

                ### Reset gradient
                self.optimizer.zero_grad()

                ### Encode the input
                x1 = self.net(x, mask_prob=0.75)
                x2 = self.net(add_noise(x), mask_prob=1.0)

                ### Calculate loss and backward pass
                loss = self.loss_fn(
                    x1.reshape(self.batch_size, -1),
                    x2.reshape(self.batch_size, -1),
                )
                loss.backward()

                ### Upgrade gradient
                self.optimizer.step()

                total_loss += loss.item()
                n_epoch_iters += 1
                self.n_iters += 1

            if interrupted: break

            total_loss /= n_epoch_iters
            loss_log.append(total_loss)

            ### Print training loss
            if verbose: print(f'Epoch {self.n_epochs}: loss = {total_loss}')

            self.n_epochs += 1

        return loss_log

    def encode(
        self,
        data,
        batch_size=None,
        mask=None,
        pooling='max',
    ):
        ''' Input: [B, C, T]
        '''
        assert self.net is not None, 'Please train or load a model'
        assert data.ndim == 3

        if batch_size is None: batch_size = self.batch_size
        n_samples, _, seq_len = data.shape

        org_training = self.net.training
        self.net.eval()

        ### Dataset and DataLoader
        dataset = TensorDataset(torch.from_numpy(data).float())
        dataloader = DataLoader(dataset, batch_size=batch_size)

        ### encode data
        with torch.no_grad():
            output = []
            for x in dataloader:
                output.append(self._eval_with_pooling(x[0], mask, pooling))
            output = torch.cat(output, dim=0).squeeze(2)
        self.net.train(org_training)
        return output.numpy()

    def _eval_with_pooling(self, x, mask=None, pooling='max'):
        ''' Input: [B, C, T]
        '''
        out = self.net(x.to(self.device, non_blocking=True), mask)
        seq_len = out.size(2)
        if pooling == 'max':
            out = F.max_pool1d(out, kernel_size=seq_len)
        elif pooling == 'avg':
            out = F.avg_pool1d(out, kernel_size=seq_len)
        elif pooling == 'max_avg' or pooling == 'avg_max':
            out_max = F.max_pool1d(out, kernel_size=seq_len)
            out_avg = F.avg_pool1d(out, kernel_size=seq_len)
            out = torch.cat([out_max, out_avg], dim=1)
        elif pooling == 'max+avg' or pooling == 'avg+max':
            out_max = F.max_pool1d(out, kernel_size=seq_len)
            out_avg = F.avg_pool1d(out, kernel_size=seq_len)
            out = out_max + out_avg
        else:
            # default to max pooling
            out = F.max_pool1d(out, kernel_size=seq_len)
        return out.cpu()


class MLP(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        return self.fc2(torch.relu(self.fc1(x)))

### training

In [ ]:
# original shape: [timestep, location, feature]
# reshape to:     [location, feature, timestep]
# [B, C, T]
pems = load_pems('data/pems04.npz')
X_train, X_val, X_test = get_train_test_splits(pems)

l = 12 * 24 * 7
X_train = split_time_series(X_train, l)

print(X_train.shape)

In [ ]:
encoder_params = {
    'hidden_dim': 64,
    'output_dim': 256,
    'depth': 6,
    'kernel_size': 3,
    'dropout': 0.2,
    'mask_mode': 'b',
}
model_params = {
    'loss_fn': InfoNCE(temperature=0.07),
    'optimizer': torch.optim.Adam,
    'train_dataset': TSTrainDataset(X_train),
    'lr': 0.0001,
    'batch_size': 16,
    'device': 'cpu',
}

encoder = CausalCNNEncoder(input_dim=X_train.shape[1], **encoder_params)
model = TSEncoder(encoder=encoder, **model_params)

log = model.fit(
    n_epochs=10,
    verbose=True,
)

### test stuff

In [ ]:
l = 12 * 24
t = 12

X_train_repr = model.encode(X_test[:, :, :l-t], pooling='avg')
X_test_repr = model.encode(X_test[:, :, :l], pooling='avg')

y_train = X_test[:, 0, l-t:l]
y_test = X_test[:, 0, l:l+t]

In [ ]:
mlp_model = MLP(256, 64, t)
loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(mlp_model.parameters(), lr=0.001)

n_epochs = 200
for epoch in range(n_epochs):
    optimizer.zero_grad()
    y_pred = mlp_model(torch.tensor(X_train_repr).float())
    loss = torch.sqrt(loss_fn(y_pred, torch.tensor(y_train).float()))
    loss.backward()
    optimizer.step()
    if (epoch+1) % 100 == 0:
        print(f'Epoch {epoch+1}, Loss: {loss.item():.4f}')

mlp_model.eval()
with torch.no_grad():
    y_pred = mlp_model(torch.tensor(X_test_repr).float())

print()
rmse = root_mean_squared_error(y_test, y_pred.numpy())
mae = mean_absolute_error(y_test, y_pred.numpy())
mape = mean_absolute_percentage_error(y_test, y_pred.numpy())
print(f'RMSE: {rmse}')
print(f'MAE: {mae}')
print(f'MAPE: {mape}')